# Modeling Demand for Cars with the Similarity Model

In this notebook, we will explore the dataset used in
Goldberg & Verboven (2005). We will estimate the Similarity Model
model given the available data using the functions defined below.

In [6]:
import numpy as np
import pandas as pd 
import os
from numpy import linalg as la
from scipy import optimize
from IPython import display
from matplotlib import pyplot as plt
import itertools as iter

# Files
import Logit_file as logit

Data
====

The dataset consists of approximately 110 vehicle makes per year in the period 1970-1999 in five european markets (Belgium, France, Germany, Italy, and the United Kingdom). Furthermore, the data contains information on various characteristics of the makes such as sales, prices, horse power, weight and other physical car characteristics. Also these characteristics may vary across markets. 

A observation in our analysis will be a market in a given year such that e.g. the French car market in 1995 counts as a single observation. If $Y = 30$ is the number of years, and $M = 5$ is the number of country-level markets, we thus have $T=Y\cdot M = 150$ markets and observations. In addition, since the available vehicle makes vary across time and place, let $\mathcal{J}_t$ denote the set of available makes in each market $t=1,\ldots,T$, and let $\mathcal{J} := \bigcup_{t=1}^T \mathcal{J}_t$ be the set of all makes which were available in some market. Then $J:=\#\mathcal{J}$ is the number of makes which were available at some point of time in the period in at least one country-level market. In our dataset there are $J = 356$ unique vehicle makes.

Our dataset includes 47 variables in total. The first three columns are market and product codes for the year, country, and make. Another variable is quantity sold (No. of new registrations) which will be used in computing observed market shares. The remaining 43 variables are potential explanatory variables. We will only consider the subset of these which describes car characteristics such as brand, after-tax price, horse power, etc. which adds up to $K=23$ characteristics. The remaining 20 variables are mainly macroeconomic variables such as e.g. GDP per capita which have been used to construct estimates of e.g. the average wage income and purchasing power. Since we are only interested in utility-shifting variables, we will not consider the latter columns. 

Reading in the dataset `eurocars.csv` we thus have a dataframe of $\sum_{t=1}^T \#\mathcal{J}_t = 11459$ rows and $47$ columns. The `ye` column runs through $y=70,\ldots,99$, the `ma` column runs through $m=1,\ldots,M$, and the ``co`` column takes values $j\in \mathcal{J}$. 

Because we consider a country-year pair as the level of observation, we construct a `market` column taking values $t=1,\ldots,T$. We also construct a `market_share` variable giving us the market share of any product $j$ in any market $t$; this will obviously take values in $[0,1]$. To deal with the fact that choice sets $\mathcal{J}_t$ vary across markets, we expand the dataframe so that every car $j\in \mathcal{J}$ which was observed in some market $t$ is in the choice set of all other markets as well, i.e. we impose $\mathcal{J}_t = \mathcal{J}$ for all markets $t$. We then impute a market share of $q_{jt}=0$ for any car $j$ which in reality was not available in market $t$. To this end we first construct an outside option $j=0$ in each market $t$  of not buying a car by letting the 'sales' of $j=0$ being determined as 

$$\mathrm{sales}_{0t} = \mathrm{pop}_t - \sum_{j=1}^J \mathrm{sales}_{jt}$$

where $\mathrm{pop}_t$ is the total population in market $t$.

We also read in the variable description of the dataset contained in `eurocars.dta`. We will use the list `x_vars` throughout to work with our explanatory variables.

Lastly, we access the underlying 3-dimensional numpy array of the explonatory variables `x` by sorting on `market` and then `co`, and subsequently resizing the explanatory variables as

> `x = dat[x_vars].values.resize((T,J,K))`

In [7]:
# Load dataset and variable names
os.chdir('../GREENCAR_notebooks/')
input_path = os.getcwd() # Assigns input path as current working directory (cwd)
descr = (pd.read_stata('eurocars.dta', iterator = True)).variable_labels()
dat = pd.read_csv(os.path.join(input_path, 'eurocars.csv'))

In [19]:
pd.DataFrame(descr, index=['description']).transpose().reset_index().rename(columns={'index' : 'variable names'})

,variable names,description
0,ye,year (=first dimension of panel)
1,ma,market (=second dimension of panel)
2,co,model code (=third dimension of panel)
3,zcode,alternative model code (predecessors and succe...
4,brd,brand code
5,type,name of brand and model
6,brand,name of brand
7,model,name of model
8,org,"origin code (demand side, country with which c..."
9,loc,"location code (production side, country where ..."


We now clean the data to fit our setup

In [9]:
### First we create the 'market' column 

dat = dat.sort_values(by = ['ye', 'ma'], ascending = True)
market_vals = [*iter.product(dat['ye'].unique(), dat['ma'].unique())]
market_vals = pd.DataFrame({'year' : [val[0] for val in market_vals], 'country' : [val[1] for val in market_vals]})
market_vals = market_vals.reset_index().rename(columns={'index' : 'market'})
dat = dat.merge(market_vals, left_on=['ye', 'ma'], right_on=['year', 'country'], how='left')

In [10]:
### Second we expand the dataset such that all cars are at least vacuously available in all markets

product_vals = [*iter.product(dat['market'].unique(), dat['co'].unique())]
product_vals = pd.DataFrame({'market' : [val[0] for val in product_vals], 'co' : [val[1] for val in product_vals]})
dat = product_vals.merge(dat, on=['market','co'], how='outer')
dat['qu'] = np.where(dat['qu'].isna(), 0, dat['qu'])

In [ ]:
### Third we construct an outside option for each market t

={key: 0 for key in dat.keys()}

In [11]:
### Fourth we compute market shares for each product j in each market t 



In [8]:
x_vars =  [dat.keys()[k] for k in [*range(5,13), *range(14,29)]] 
x_lab = (pd.DataFrame(descr, index=['description'])[x_vars].transpose().reset_index().rename(columns={'index' : 'variable names'}))

,variable names,description
0,type,name of brand and model
1,brand,name of brand
2,model,name of model
3,org,"origin code (demand side, country with which c..."
4,loc,"location code (production side, country where ..."
5,cla,class or segment code
6,home,domestic car dummy (appropriate interaction of...
7,frm,firm code
8,cy,cylinder volume or displacement (in cc)
9,hp,horsepower (in kW)


In [15]:
len(dat[dat.qu.isna()==True].qu)

0

In [16]:
dat.qu

0         2700.0
1         3500.0
2         6700.0
3         7800.0
4         9500.0
          ...   
53395        0.0
53396     8036.0
53397    17119.0
53398     4642.0
53399     7638.0
Name: qu, Length: 53400, dtype: float64

In [17]:
dat

,market,co,ye,ma,zcode,brd,type,brand,model,org,...,tax,pop,ngdp,rgdp,engdp,ergdp,engdpc,ergdpc,year,country
0,0,15,70.0,1.0,14.0,2.0,audi 100/200,audi,100/200,2.0,...,"0,25",9660000.0,1280999948288,3940724506624,2.579524e+10,7.935358e+10,"2670,314697","8214,65625",70.0,1.0
1,0,26,70.0,1.0,35.0,4.0,citroen 2 CV 6 - 2 CV 4,citroen,2CV6,1.0,...,"0,25",9660000.0,1280999948288,3940724506624,2.579524e+10,7.935358e+10,"2670,314697","8214,65625",70.0,1.0
2,0,36,70.0,1.0,36.0,4.0,citroen dyane,citroen,dyane,1.0,...,"0,25",9660000.0,1280999948288,3940724506624,2.579524e+10,7.935358e+10,"2670,314697","8214,65625",70.0,1.0
3,0,64,70.0,1.0,67.0,7.0,fiat 128,fiat,128,3.0,...,"0,25",9660000.0,1280999948288,3940724506624,2.579524e+10,7.935358e+10,"2670,314697","8214,65625",70.0,1.0
4,0,71,70.0,1.0,80.0,8.0,ford escort,ford,escort,2.0,...,"0,25",9660000.0,1280999948288,3940724506624,2.579524e+10,7.935358e+10,"2670,314697","8214,65625",70.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53395,149,962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53396,149,977,99.0,5.0,229.0,23.0,rover 75,rover,75,7.0,...,"0,174999997",58200000.0,889870024704,NaN,1.052245e+12,NaN,"18079,80273",NaN,99.0,5.0
53397,149,978,99.0,5.0,287.0,29.0,toyota yaris,toyota,yaris,4.0,...,"0,174999997",58200000.0,889870024704,NaN,1.052245e+12,NaN,"18079,80273",NaN,99.0,5.0
53398,149,979,99.0,5.0,306.0,30.0,volkswagen lupo,VW,lupo,2.0,...,"0,174999997",58200000.0,889870024704,NaN,1.052245e+12,NaN,"18079,80273",NaN,99.0,5.0
